In [1]:
import numpy as np
import pandas as pd
from pathmgmt import pathmgmt as myPath
import os

In [19]:
# Combine all alphas
def get_stat():
    res = pd.DataFrame()
    for folderName in os.listdir(myPath.PLOT_DIR):
        if 'zz9999' not in folderName or 'Mfl' not in folderName:
            continue
        # print(folderName)
        folder = myPath.PLOT_DIR/folderName/'statistics'
        for fileName in os.listdir(folder):
            # print(fileName)
            if 'Rank' in fileName:
                continue
            file = folder/fileName
            df = pd.read_csv(file)
            df.index = [f'{folderName};{fileName[:-15]}']
            # print(df)
            if res.empty:
                res = df
            else:
                res = pd.concat([res, df])
    return res

In [20]:
stats = get_stat()

In [21]:
stats.describe()

,Annualized Return,Annualized Excess Return,IR,IR long only,IC,Max Drawdown,Max Drawdown long only,daily Turnover
count,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000,570.000000
mean,0.162081,0.001690,2.327642,0.037825,0.009827,0.075129,0.203194,0.119939
std,0.110396,0.074200,1.231335,0.670801,0.006236,0.029095,0.118852,0.181906
min,-0.044523,-0.371735,-0.892053,-2.616625,-0.001029,0.001949,0.074371,0.007288
25%,0.072708,-0.043306,1.369118,-0.402924,0.004612,0.060239,0.132547,0.024152
50%,0.153105,0.013792,2.276586,0.131970,0.009169,0.073543,0.155987,0.060350
75%,0.232277,0.057978,3.314184,0.600609,0.014036,0.087169,0.247059,0.136977
max,0.570503,0.120719,6.109713,1.078943,0.030022,0.174114,1.073902,1.440506


In [50]:
good_alphas = list(stats.loc[(stats.IR > 2.5) & (stats['daily Turnover'] < 0.2)].index)

In [51]:
len(good_alphas)

227

In [52]:
# combine daily pnl
def get_pnl():
    res = pd.DataFrame()
    for folderName in os.listdir(myPath.PLOT_DIR):
        if 'zz9999' not in folderName or 'Mfl' not in folderName:
            continue
        # print(folderName)
        folder = myPath.PLOT_DIR/folderName/'PnL_results'
        for fileName in os.listdir(folder):
            file = folder/fileName
            if 'Rank' in fileName:
                continue
            df = pd.read_csv(file)
            # df = df.set_index('time')
            # df = df['pnl']
            df = df[['time', 'pnl']]
            df.rename(
                columns={'pnl': f'{folderName};{fileName[:-13]}'}, inplace=True)
            if res.empty:
                res = df
            else:
                res = res.merge(df, on='time', how='outer')
    res = res.set_index('time').sort_index()
    return res


In [53]:
pnls = get_pnl()

In [54]:
corr = pnls[good_alphas].corr()

In [63]:
from itertools import combinations
import random
res = []
num = 6
good_candidates = list(
    ((corr.mask(np.eye(len(corr), dtype=bool)).abs() < 0.6).sum() > num).index)
random.shuffle(good_candidates)

In [64]:
len(good_candidates)

227

In [65]:
for cols in combinations(good_candidates, num):
    corr_small = corr.loc[cols, cols]
    if (~(corr_small.mask(np.eye(len(corr_small), dtype=bool)).abs() > 0.6).any()).sum() == num:
        res.append(corr_small)
        break

In [66]:
res[0]

,alpha.YaoMfl017-20days-zz9999;20180101-20201231-None-Industry-holding5days,alpha.YaoMfl004-3days-zz9999;20180101-20201231-None-Industry-holding20days,alpha.YaoMfl005-5days-zz9999;20180101-20201231-None-Industry-holding1days,alpha.YaoMfl008-60days-zz9999;20180101-20201231-None-Industry-holding1days,alpha.YaoMfl005-60days-zz9999;20180101-20201231-None-Industry-holding60days,alpha.YaoMfl012-5days-zz9999;20180101-20201231-None-Industry-holding20days
alpha.YaoMfl017-20days-zz9999;20180101-20201231-None-Industry-holding5days,1.000000,0.378784,0.354229,0.143439,-0.010634,0.141658
alpha.YaoMfl004-3days-zz9999;20180101-20201231-None-Industry-holding20days,0.378784,1.000000,0.482321,0.445255,0.473882,-0.194511
alpha.YaoMfl005-5days-zz9999;20180101-20201231-None-Industry-holding1days,0.354229,0.482321,1.000000,0.496853,0.572562,0.359821
alpha.YaoMfl008-60days-zz9999;20180101-20201231-None-Industry-holding1days,0.143439,0.445255,0.496853,1.000000,0.482728,0.114442
alpha.YaoMfl005-60days-zz9999;20180101-20201231-None-Industry-holding60days,-0.010634,0.473882,0.572562,0.482728,1.000000,-0.013655
alpha.YaoMfl012-5days-zz9999;20180101-20201231-None-Industry-holding20days,0.141658,-0.194511,0.359821,0.114442,-0.013655,1.000000


In [78]:
# stats.loc[stats.index.isin(res[0].index)]

In [68]:
# max_IR = float('-inf')
# max_idx = -1
# for i in range(len(res)):
#     total_IR = np.sum([(stats.loc[stats.index==name]['IR long only']) for name in res[i].index])
#     if total_IR > max_IR:
#         max_IR = total_IR
#         max_idx = i

In [69]:
# res[max_idx]

In [70]:
# stats.loc[stats.index.isin(res[max_idx].index)]

In [71]:
stats.loc[stats.index.isin(res[0].index)]


,Annualized Return,Annualized Excess Return,IR,IR long only,IC,Max Drawdown,Max Drawdown long only,daily Turnover
alpha.YaoMfl004-3days-zz9999;20180101-20201231-None-Industry-holding20days,0.233410,0.076358,2.885914,0.779023,0.015435,0.072570,0.127085,0.035626
alpha.YaoMfl005-5days-zz9999;20180101-20201231-None-Industry-holding1days,0.246623,0.046091,3.708832,0.451559,0.015797,0.048168,0.146284,0.188511
alpha.YaoMfl005-60days-zz9999;20180101-20201231-None-Industry-holding60days,0.156576,0.056910,3.402267,0.628960,0.009910,0.038853,0.088256,0.010387
alpha.YaoMfl008-60days-zz9999;20180101-20201231-None-Industry-holding1days,0.141530,0.003864,2.822629,0.039789,0.009041,0.068625,0.153522,0.022877
alpha.YaoMfl012-5days-zz9999;20180101-20201231-None-Industry-holding20days,0.012000,0.024485,2.530231,0.133326,0.006377,0.004297,0.310071,0.069073
alpha.YaoMfl017-20days-zz9999;20180101-20201231-None-Industry-holding5days,0.251634,0.016917,3.584042,0.169992,0.015659,0.071439,0.155082,0.131062
